# DSCoV Napari Workshop

Welcome to DSCoV! Today we are going to explore how to use Napari to view 3D volumetric images and visualize the results of image segmentation. We will also learn how to customize the GUI of Napari for your specific tasks.

Please feel free to execute the cells in this notebook and follow along.

Special thanks to **Dr. Alexander Fleischmann** and **Sara Zeppilli** for providing the 3D volumetric images in their research as examples for this workshop!

# Task: Visualizing Image Segmentation results

## Task description

We have 3D microscopic scans of mice brains at different developmental stages. They have been cleared and stained with different antibiotics to reveal different cell nuclei in the allocortex and neocortex regions in the brain. We have run image segmentation to identify the cell nuclei. Now we need to visualize the prediction and compare it with the ground truth to understand how well the segmentation performs.

## Load package and images

First, we load the `napari` package and others used in this tutorial.

In [ ]:
# Load the necessary packages

import napari
import numpy as np
from skimage.io import imread

Then we load the image, ground truth, and prediction into memory as numpy arrays

In [ ]:
# The image is a 3D 16-bit tif file which we will load with `imread` from scikit-image
brain_image = imread("images/brain_image.tif")

# Ground truth and predictions are just numpy arrays saved in .npy format
ground_truth = np.load("images/ground_truth.npy").astype(int)
prediction = np.load("images/prediction.npy").astype(int)

print("Shape of image: ", brain_image.shape)
print("Shape of ground truth: ", ground_truth.shape)
print("Shape of prediciton: ", prediction.shape)

## View the 3D volumetric image

We can pass the loaded image to `napari.view_image()`. A window will pop up with the image.

Under the hood, this function creates an `image` layer in a `Viewer` object which we can manipulate later to add more layers. Any changes we make to this `Viewer` object will be reflected in the napari window immediately. Let's take a look at the window itself.

![napari viewer window](https://napari.org/stable/_images/viewer_layout.jpg)

In [ ]:
viewer: napari.Viewer = napari.view_image(brain_image)

This image stack is visualized in the **canvas** region of the viewer window. You can use the **dimension sliders** to view each slice on the z-axis one after another.

The six viewer buttons that you can use:
* **IPython console**: if you launch Napari from command line, you wil have an interactive IPython console to interact with the `viewer` object in memory.
* **ndview**: toggle 2D/nD view of the image
* **Change order of axes**: view the image from different angles
* **Transpose**: transpose (rotate) the image
* **Grid view**: view layers separately in a grid
* **Reset view**: reset all changes and go back to the original state

Feel free to use different buttons and see what happens. You can always use the last one to go back to the original state.

## Adding layers

The most important concept in `napari` is `layers`. You can add different types of layers on top of each other. There are 7 types of layers:

* `image` layer: Displays an image. [More info](https://napari.org/stable/howtos/layers/image.html)
* `labels` layer: An array of type `int`. Often used to display segmentation results, with different `int` values representing the class of the object in each segment. [More info](https://napari.org/stable/howtos/layers/labels.html)
* `points` layer: An N x D array with `n` points in `d` dimensions. [More info](https://napari.org/stable/howtos/layers/points.html)
* `shapes` layer: A list of `k` N x D arrays with `k` shapes of `n` points in `d` dimensions. [More info](https://napari.org/stable/howtos/layers/shapes.html)
* `surface` layer: A generalization of shapes layer to display polygons. [More info](https://napari.org/stable/howtos/layers/surface.html)
* `tracks` layer: used to describe tracks of objects across time steps. [More info](https://napari.org/stable/howtos/layers/tracks.html)
* `vectors` layer: used to display vectors. [More info](https://napari.org/stable/howtos/layers/vectors.html)

We can use `viewer.add_*()` function to add a layer to the viewer. This method will return an object of `Layer` type which you can modify later.

In [ ]:
gt_layer = viewer.add_labels(ground_truth)

Now that we have added a layer to the viewer. However, the layer does not look too good right now. We can customize the layer with **layer controls** on the left, once the layer is selected in the **layer list**. However, we can also modify the layers programmatically.

In [ ]:
# Let's create a dict that maps certain values in the layer to a set color.

colormap = {
    1: (0, 1, 0, 0.55) # An RGBA 4-tuple
}

gt_layer.color = colormap
gt_layer.name = "Ground Truth"

We can also add all these settings while adding the layer:

In [ ]:
pred_layer = viewer.add_labels(prediction, name="Predictions", color={1: (1, 0, 1, 0.5)})

## Customizing the interface

One of the biggest advantage of `napari` is its flexibility and extensibility. You can customize its GUI to add buttons, controls, and have it respond to custom mouse, keyboard events. Let us begin with a simple one: key binding.

### Event handling in `napari`: key binding

This is done through a decorator

In [ ]:
@viewer.bind_key("d")
def delete_layer(viewer): # The function being decorated accepts a Viewer object as argument
    deleted_layer = viewer.layers.pop(-1)
    
    viewer.status = f"Layer {deleted_layer.name} removed."

### We can also have napari respond to mouse events

`napari` supports various mouse events from clicking, double clicking, to dragging

In [ ]:
empty_layer = viewer.add_image(np.zeros(brain_image.shape), name="random noise")

@empty_layer.mouse_drag_callbacks.append
def randomize(layer, event):
    layer.data = np.random.random(brain_image.shape)

### Now, let's try adding additional functionalities to Napari

It seems that the image is a bit too dark. Aside from adjusting the `contrast_limit` option, we can also use any arbitrary Python image processing library to process the image. Here we are going to use `scikit-image` to perform automatic histogram based adjustments to enhance our image. There are both global and local adjustment options available to us. For details on these adjustments, please check [this page](https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_local_equalize.html#sphx-glr-auto-examples-color-exposure-plot-local-equalize-py).

`Napari` uses `magicgui` for customizing its GUI. `magicgui` checks the type annotations of the function being decorated to determine which control to show in the widget.

In the next example, we use type annotations of the parameter types and return types of `adjustment_widget` function to tell `magicgui` what to show in the widget dock.

``` python
@magicgui(
    call_button="Apply threshold",
    adjustment={"widget_type": "RadioButton", "choices": ["global", "local"]},
    radius={"widget_type": "Slider", "max": 10}
)
def adjustment_widget(
    image: napari.layers.Image, 
    adjustment: str="global", 
    radius:int=3
) -> napari.layers.Image:
```

* `radius` is `int` type, we tell `magicgui` to display a slider to choose the value for `radius`
* `adjustment` is `str` type. By default, `magicgui` will display a textbox. However, what we want is a set of radio buttons, so we specify the `widget_type` to be `RadioButton` with two choices.
* You can use `napari` builtin types too. Here we are using `Image`, which is a subclass of `Layer` type. Napari will show a dropdown of all layers of `Image` type. The argument passed to the function will be a reference to that layer.
* We also specify the return type of the function. If the return type is a subclass of `Layer`, then `napari` will add this layer to the viewer.
* `call_button` parameter to the decorator produces a button. Once clicked, it will execute the function, passing all values in the widget to the function.

In [ ]:
from magicgui import magicgui
from skimage import exposure
from skimage.morphology import ball
from skimage.filters import rank

@magicgui(
    call_button="Apply enhancement",
    adjustment={"widget_type": "RadioButton", "choices": ["global", "local"]},
    radius={"widget_type": "Slider", "max": 10}
)
def adjustment_widget(
    image: napari.layers.Image, 
    adjustment: str="global", 
    radius:int=3
) -> napari.layers.Image:
    
    data = image.data # Extract the data from the layer
    
    if adjustment == "global":
        new_data = exposure.equalize_hist(data)
    elif adjustment == "local":
        neighborhood = ball(radius)
        new_data = rank.equalize(data, footprint=neighborhood)
        
    return napari.layers.Image(new_data, name=f"{adjustment} adjusted")

# Register this widget to the viewer
viewer.window.add_dock_widget(adjustment_widget)

A "Dock Widget" will show up. Now pick an adjustment (global or local), and if you choose local adjustment, choose a radius of the neighborhood (this might be slow), then click **Apply enhancement**.

You can run any arbitrary code within the `adjustment_widget` function. That means you can do absolutely anything with your image. For example, you can run an end-to-end image segmentation/object detection network and have the results visualized immediately. Even better, once you have established a workflow, you can publish it as a `napari` plugin. You can install plugins directly from the `plugins` menu. Or you can go to [napari hub](https://www.napari-hub.org/) to download more cutting-edge plugins.

## Use `napari` to view large volumes that do not fit into memory

Volumetric images are usually so large that they do not fit into the memory of a single computer. `napari` works not only with `numpy` arrays, it also works with arrays in library such as `dask`, `zarr`, and `xarray`, as long as they implement an `asarray()` method that turns their data structures into `numpy` arrays.

* `dask` is a library used to perform out-of-memory computation on large data.
* `zarr` is a storage backend to store chucked arrays that can be compressed.
* `xarray` provides support for labeled multi-dimensional arrays often used in Bayesian analyses.

With `dask-image`, viewing very large multi-dimensional images can be very simple. In the above example, we used `skimage.io.imread` to read the image. `dask-image` also provides an `imread` function to lazily load images in chunks, which can then be fed to `napari`:

``` python
from dask_image import imread
image = imread("<path>/*.tif") # Assuming each slice is a numbered tif

viewer = napari.view_image(image)
```

In [ ]:
import dask_image.imread
import dask.array as da

# Lasily load the image with dask_image
very_large_image = dask_image.imread.imread("/gpfs/data/datasci/yxu150/projects/cv-segment-nuclei/data/full_images/stitch 171211_P1-Ctip2_ENS/Stitched Image_*.tif")
very_large_label = da.from_zarr("/gpfs/data/datasci/yxu150/projects/cv-segment-nuclei/data/full_labels/CTIP2_P1/candidate_2.zarr")

In [ ]:
very_large_image

In [ ]:
very_large_label

In [ ]:
large_image_viewer = napari.view_image(
    very_large_image, 
    contrast_limits=[0, 5000]
)
large_label_layer = large_image_viewer.add_labels(
    very_large_label > 0, 
    name="Predictions", 
    color={True: (0, 1, 0, 0.5)}
)

# Bonus: Use `Napari` with Dall-E

OpenAI just published their Dall-E API. We can use this API to write a Dall-E image generation interface in `Napari`. Here's a very simple example. You can register at OpenAI to get your own API key to play with it yourself.

In [ ]:
import os
import openai

openai.api_key = os.getenv("OPENAI_KEY") # Replace this with your own key

In [ ]:
viewer = napari.view_image(np.zeros((1024, 1024, 3)), name="Generated Image")

In [ ]:
@magicgui(
    call_button="Generate Image"
)
def generate_image(image_data: napari.types.ImageData, prompt:str) -> napari.types.ImageData:
    if image_data is not None and prompt != "":
        response = openai.Image.create(
            prompt=prompt,
            size="1024x1024",
            n=1
        )
        return imread(response["data"][0]["url"])
    
viewer.window.add_dock_widget(generate_image)